# NYC Real Estate — DNNs, SHAP & Aleatoric Uncertainty (Notebook 2)

**Runs right after the cleaning phase.** Follows the professor's workflow: two-headed model (μ + σ²), MC Dropout, Gaussian & Empirical coverage, SHAP for Price / Epistemic / Aleatoric, Trust Map quadrants, and surgical waterfalls.

## Prerequisite: Load cleaned data

Either run **notebook copy.ipynb** through the cleaning phase (through the price-filter cell) so `df` is in memory, or save the cleaned dataframe to `data/cleaned_df.parquet` (or `.csv`) and load it in the next cell.

In [1]:
import os
import numpy as np
import pandas as pd
import random
if os.environ.get("CI"):
    import matplotlib
    matplotlib.use("Agg")
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
import shap
import warnings
warnings.filterwarnings("ignore")

# Load df if saved; otherwise assume df is in memory (run notebook 1 first)
try:
    os.makedirs("data", exist_ok=True)
    if os.path.exists("data/cleaned_df.parquet"):
        df = pd.read_parquet("data/cleaned_df.parquet")
        print(f"Loaded df from data/cleaned_df.parquet: {len(df):,} rows")
    elif os.path.exists("data/cleaned_df.csv"):
        df = pd.read_csv("data/cleaned_df.csv")
        print(f"Loaded df from data/cleaned_df.csv: {len(df):,} rows")
    else:
        raise FileNotFoundError("No saved cleaned df")
except FileNotFoundError:
    try:
        _ = len(df)
        print(f"Using df from memory: {len(df):,} rows")
    except NameError:
        raise RuntimeError(
            "No cleaned dataframe. Run notebook copy.ipynb through the cleaning phase, "
            "or save cleaned df to data/cleaned_df.parquet (or .csv)."
        )
df.head(2)

/Users/dmitrii/Desktop/advanced_dl_project/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: No cleaned dataframe. Run notebook copy.ipynb through the cleaning phase, or save cleaned df to data/cleaned_df.parquet (or .csv).

## Set seeds (reproducibility)

In [ ]:
def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.config.experimental.enable_op_determinism()

set_seeds(42)

## Feature engineering (ensure sub_region & affluence_score exist)

If your cleaning already added these, this is a no-op; otherwise we create them so the same feature set works.

In [ ]:
if "sub_region" not in df.columns:
    df["sub_region"] = df["BOROUGH"].astype(str) + "_" + df["NEIGHBORHOOD"].str[:30]
if "affluence_score" not in df.columns:
    df["affluence_score"] = pd.qcut(df["SALE PRICE"].rank(method="first"), q=10, labels=False, duplicates="drop")
    df["affluence_score"] = df["affluence_score"].fillna(0).astype(int)

borough_dummies = pd.get_dummies(df["BOROUGH"].astype(str), prefix="borough")
sub_region_dummies = pd.get_dummies(df["sub_region"].astype(str), prefix="sub_region")
X_df = pd.concat([
    df[["GROSS SQUARE FEET", "BUILDING_AGE", "affluence_score"]],
    borough_dummies,
    sub_region_dummies
], axis=1)
feature_names = X_df.columns.tolist()
feature_cols = feature_names  # professor-style name
X = X_df.values

y_raw = df["SALE PRICE"].values
y = np.log(y_raw)

X_train, X_test, y_train, y_test, y_train_raw, y_test_raw = train_test_split(
    X, y, y_raw, test_size=0.2, random_state=42
)

scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

print(f"Features: {X_train.shape[1]}, Train: {len(X_train)}, Test: {len(X_test)}")
print(f"log(price) range: [{y_train.min():.2f}, {y_train.max():.2f}]")

## Two-headed model (μ + σ²) and Aleatoric loss

Gaussian NLL in log space; MC Dropout kept at inference for epistemic uncertainty.

In [ ]:
def aleatoric_loss(y_true, y_pred):
    mu = y_pred[:, 0:1]
    sigma_sq = y_pred[:, 1:2]
    epsilon = 1e-6
    return tf.reduce_mean(
        0.5 * tf.math.log(sigma_sq + epsilon) +
        0.5 * tf.math.divide(tf.square(y_true - mu), sigma_sq + epsilon)
    )

def make_combined_model(n_features):
    inputs = Input(shape=(n_features,))
    x = Dense(128, activation="relu")(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x, training=True)
    x = Dense(64, activation="relu")(x)
    x = BatchNormalization()(x)
    mu = Dense(1, name="mu")(x)
    sigma_sq = Dense(1, activation="softplus", name="sigma_sq")(x)
    return tf.keras.Model(inputs=inputs, outputs=Concatenate()([mu, sigma_sq]))

model = make_combined_model(X_train.shape[1])
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=aleatoric_loss)
model.summary()

## Train and plot learning curve

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=32,
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=15, restore_best_weights=True)],
    verbose=1
)

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(history.history["loss"], label="Train NLL", color="teal", lw=2)
ax.plot(history.history["val_loss"], label="Val NLL", color="orange", lw=2)
ax.set_title("Learning Curve (Negative Log-Likelihood)", fontsize=14, fontweight="bold")
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## MC Dropout inference: epistemic + aleatoric in dollars

In [ ]:
T = 100
preds = np.stack([model(X_test, training=True).numpy() for _ in range(T)])
all_mu_log = preds[:, :, 0]
all_sigma_sq_log = preds[:, :, 1]

mu_mean_log = all_mu_log.mean(axis=0)
mu_mean_dollars = np.exp(mu_mean_log)

epistemic_var_log = all_mu_log.var(axis=0)
epistemic_std_log = np.sqrt(epistemic_var_log)
epistemic_std_dollars = mu_mean_dollars * epistemic_std_log

aleatoric_var_log = all_sigma_sq_log.mean(axis=0)
aleatoric_std_log = np.sqrt(aleatoric_var_log)
aleatoric_std_dollars = mu_mean_dollars * aleatoric_std_log

total_std_dollars = np.sqrt(epistemic_std_dollars**2 + aleatoric_std_dollars**2)

r2 = r2_score(y_test_raw, mu_mean_dollars)
print(f"R2 (dollars): {r2:.4f}")
print(f"Epistemic std: mean=${epistemic_std_dollars.mean():,.0f}, max=${epistemic_std_dollars.max():,.0f}")
print(f"Aleatoric std:  mean=${aleatoric_std_dollars.mean():,.0f}, max=${aleatoric_std_dollars.max():,.0f}")

## Visualization: scatter (colored by uncertainty) + uncertainty decomposition

In [ ]:
fig, (ax2, ax3) = plt.subplots(1, 2, figsize=(14, 5))

sc = ax2.scatter(y_test_raw, mu_mean_dollars, c=total_std_dollars, cmap="magma", alpha=0.5, s=15)
ax2.plot([y_test_raw.min(), y_test_raw.max()], [y_test_raw.min(), y_test_raw.max()], "r--", lw=2)
plt.colorbar(sc, ax=ax2, label="Total Uncertainty ($)")
ax2.set_title(f"Test Performance (R2: {r2:.3f})", fontsize=14, fontweight="bold")
ax2.set_xlabel("Actual Price ($)")
ax2.set_ylabel("Predicted Price ($)")

ax3.hist(epistemic_std_dollars, bins=30, alpha=0.5, label="Epistemic (Model Doubt)", color="blue")
ax3.hist(aleatoric_std_dollars, bins=30, alpha=0.5, label="Aleatoric (Data Noise)", color="red")
ax3.set_title("Where does the doubt come from?", fontsize=14, fontweight="bold")
ax3.set_xlabel("Standard Deviation ($)")
ax3.legend()
plt.tight_layout()
plt.show()

## Point-wise coverage: Gaussian (per-pass 95% capture)

In [ ]:
sigma_samples_log = np.sqrt(all_sigma_sq_log)
individual_hit_mask = (
    (y_test_raw >= np.exp(all_mu_log - 1.96 * sigma_samples_log)) &
    (y_test_raw <= np.exp(all_mu_log + 1.96 * sigma_samples_log))
)
house_hit_rates = np.mean(individual_hit_mask, axis=0)
print(f"Average Gaussian hit rate across houses: {np.mean(house_hit_rates):.2%}")

df_trend = pd.DataFrame({"price": y_test_raw, "hit_rate": house_hit_rates}).sort_values("price")
df_trend["rolling_avg"] = df_trend["hit_rate"].rolling(window=150, center=True).mean()

plt.figure(figsize=(12, 6))
plt.scatter(y_test_raw, house_hit_rates, alpha=0.3, c=house_hit_rates, cmap="RdYlGn", s=15)
plt.plot(df_trend["price"], df_trend["rolling_avg"], color="blue", lw=3, label="Local Reliability Trend")
plt.axhline(0.95, color="black", linestyle="--", alpha=0.6, label="95% Target")
plt.title("Gaussian Consensus: How many MC passes captured the truth?", fontsize=14)
plt.xlabel("Actual House Price ($)")
plt.ylabel("Hit Rate (Fraction of MC Passes)")
plt.ylim(-0.05, 1.05)
plt.colorbar(label="Consensus Score")
plt.legend(loc="lower left")
plt.grid(alpha=0.2)
plt.show()

## Point-wise coverage: Empirical (simulated samples, 2.5–97.5%)

In [ ]:
full_samples_dollars = np.exp(np.random.normal(loc=all_mu_log, scale=np.sqrt(all_sigma_sq_log)))
emp_lower = np.percentile(full_samples_dollars, 2.5, axis=0)
emp_upper = np.percentile(full_samples_dollars, 97.5, axis=0)
emp_hit_mask = (y_test_raw >= emp_lower) & (y_test_raw <= emp_upper)
emp_coverage = np.mean(emp_hit_mask)
print(f"Empirical 95% coverage: {emp_coverage:.2%}")
if emp_coverage < 0.90:
    print("Coverage is low; model may be overconfident.")
else:
    print("Target hit: uncertainty captures reality well.")

emp_hits = emp_hit_mask.astype(int)
jitter = np.random.uniform(-0.05, 0.05, size=len(emp_hits))
plt.figure(figsize=(12, 6))
plt.scatter(y_test_raw, emp_hits + jitter, alpha=0.2, c=emp_hits, cmap="RdYlGn", s=10)
df_rel = pd.DataFrame({"price": y_test_raw, "hit": emp_hits}).sort_values("price")
df_rel["rolling_coverage"] = df_rel["hit"].rolling(window=100, center=True).mean()
plt.plot(df_rel["price"], df_rel["rolling_coverage"], color="blue", lw=3, label="Local Coverage (Rolling Mean)")
plt.axhline(0.95, color="red", linestyle="--", label="95% Target")
plt.title("Empirical Reliability: Where do simulations fail?", fontsize=14)
plt.xlabel("Actual House Price ($)")
plt.ylabel("Hit (1) or Miss (0)")
plt.yticks([0, 1], ["Miss", "Hit"])
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## SHAP: surgical predictors (Price, Epistemic, Aleatoric)

In [ ]:
def predict_mean_final(x_np):
    preds = model(x_np, training=False).numpy()
    mu_log = preds[:, 0]
    return np.exp(mu_log)

def predict_epistemic_final(x_np):
    preds = np.stack([model(x_np, training=True).numpy() for _ in range(50)])
    all_mu_log = preds[:, :, 0]
    all_mu_dollars = np.exp(all_mu_log)
    return all_mu_dollars.std(axis=0)

def predict_aleatoric_final(x_np):
    preds = model(x_np, training=False).numpy()
    mu_log = preds[:, 0]
    sigma_sq_log = preds[:, 1].flatten()
    aleatoric_std_log = np.sqrt(sigma_sq_log)
    return np.exp(mu_log) * aleatoric_std_log

X_subset = X_test[:40]
background_vibrant = shap.sample(X_train, 100)

surgical_predictors = {
    "Price (Mean)": predict_mean_final,
    "Model Doubt (Epistemic)": predict_epistemic_final,
    "Data Noise (Aleatoric)": predict_aleatoric_final,
}

test_prices = predict_mean_final(X_subset)
print("Pulse check:", f"Sample 1: ${test_prices[0]:,.2f}", f"Sample 2: ${test_prices[1]:,.2f}", f"Std: {test_prices.std():.4f}")

## SHAP summary plots (beeswarm) for all three heads

In [ ]:
for title, func in surgical_predictors.items():
    print(f"Deep-scanning {title}...")
    explainer = shap.KernelExplainer(func, background_vibrant)
    sv = explainer.shap_values(X_subset, nsamples=500)
    if isinstance(sv, list):
        sv = sv[0]
    plt.figure(figsize=(10, 6))
    shap.summary_plot(sv, X_subset, feature_names=feature_cols, show=False)
    plt.title(f"Surgical Focus: {title}", fontsize=16, fontweight="bold")
    plt.xlabel("Impact in Dollars ($)")
    plt.tight_layout()
    plt.show()

## Dependence plots (one feature vs SHAP)

Similar to partial dependence: how does one feature drive Price, Epistemic, or Aleatoric?

In [ ]:
# Pick a numeric feature that exists in feature_names (e.g. GROSS SQUARE FEET or affluence_score)
target_feature = "GROSS SQUARE FEET" if "GROSS SQUARE FEET" in feature_names else feature_names[0]
explanations = {}
for title, func in surgical_predictors.items():
    expl = shap.KernelExplainer(func, background_vibrant)
    sv = expl.shap_values(X_subset, nsamples=500)
    if isinstance(sv, list):
        sv = sv[0]
    explanations[title] = shap.Explanation(
        values=sv, data=X_subset, feature_names=feature_cols, base_values=expl.expected_value
    )
for title, exp in explanations.items():
    if target_feature not in feature_names:
        continue
    plt.figure(figsize=(10, 6))
    shap.plots.scatter(exp[:, target_feature], color=exp, show=False)
    plt.title(f"Dependency: {target_feature} vs {title}", fontsize=14, fontweight="bold")
    plt.ylabel("SHAP Value (Impact in $)")
    plt.xlabel(f"Actual {target_feature} Value")
    plt.grid(alpha=0.2, linestyle="--")
    plt.tight_layout()
    plt.show()

## Find the most uncertain house

In [ ]:
max_unc_idx = np.argmax(total_std_dollars)
print(f"Row with highest combined uncertainty: {max_unc_idx}")
print(f"Total Std: ${total_std_dollars[max_unc_idx]:,.2f}")
print(f"  Epistemic: ${epistemic_std_dollars[max_unc_idx]:,.2f} | Aleatoric: ${aleatoric_std_dollars[max_unc_idx]:,.2f}")

## Waterfall: Price (mean) for most uncertain house

In [ ]:
explainer_waterfall = shap.KernelExplainer(predict_mean_final, background_vibrant)
shap_values_single = explainer_waterfall.shap_values(X_test[max_unc_idx : max_unc_idx + 1], nsamples=1000)
if isinstance(shap_values_single, list):
    shap_values_single = shap_values_single[0]
exp_single = shap.Explanation(
    values=shap_values_single[0],
    base_values=explainer_waterfall.expected_value,
    data=X_test[max_unc_idx],
    feature_names=feature_cols
)
plt.figure(figsize=(10, 6))
shap.plots.waterfall(exp_single, show=False)
plt.title(f"Why is House #{max_unc_idx} priced at ${predict_mean_final(X_test[max_unc_idx:max_unc_idx+1])[0]:,.0f}?", fontsize=14, fontweight="bold", pad=20)
plt.show()

## Waterfalls: Epistemic & Aleatoric for most uncertain house

In [ ]:
uncertainty_targets = {
    "Model Doubt (Epistemic)": predict_epistemic_final,
    "Data Noise (Aleatoric)": predict_aleatoric_final,
}
for title, func in uncertainty_targets.items():
    explainer = shap.KernelExplainer(func, background_vibrant)
    sv = explainer.shap_values(X_test[max_unc_idx : max_unc_idx + 1], nsamples=1000)
    if isinstance(sv, list):
        sv = sv[0]
    exp = shap.Explanation(
        values=sv[0],
        base_values=explainer.expected_value,
        data=X_test[max_unc_idx],
        feature_names=feature_cols
    )
    plt.figure(figsize=(10, 6))
    shap.plots.waterfall(exp, show=False)
    plt.title(f"Uncertainty Driver: {title} (Row {max_unc_idx})", fontsize=14, fontweight="bold", pad=20)
    plt.show()

## 2D Trust Map: SHAP magnitude vs uncertainty magnitude (quadrants)

In [ ]:
print("Generating SHAP values for all targets...")
targets = {"mean": predict_mean_final, "epistemic": predict_epistemic_final, "aleatoric": predict_aleatoric_final}
sv_results = {}
for name, func in targets.items():
    explainer = shap.KernelExplainer(func, background_vibrant)
    sv = explainer.shap_values(X_subset, nsamples=500)
    if isinstance(sv, list):
        sv = sv[0]
    sv_results[name] = sv

shap_mag = np.abs(sv_results["mean"]).sum(axis=1)
unc_mag = np.abs(sv_results["epistemic"]).sum(axis=1) + np.abs(sv_results["aleatoric"]).sum(axis=1)

quad_df = pd.DataFrame({"idx": np.arange(len(X_subset)), "shap_mag": shap_mag, "unc_mag": unc_mag})
s_mid = quad_df["shap_mag"].median()
u_mid = quad_df["unc_mag"].median()

def assign_quadrant(row):
    if row["shap_mag"] >= s_mid and row["unc_mag"] < u_mid:
        return "High SHAP, Low Unc"
    if row["shap_mag"] >= s_mid and row["unc_mag"] >= u_mid:
        return "High SHAP, High Unc"
    if row["shap_mag"] < s_mid and row["unc_mag"] < u_mid:
        return "Low SHAP, Low Unc"
    return "Low SHAP, High Unc"

quad_df["category"] = quad_df.apply(assign_quadrant, axis=1)
print(quad_df["category"].value_counts())

categories = {
    "High SHAP, Low Unc": {"color": "#1f77b4", "desc": "(Confident Signal)"},
    "High SHAP, High Unc": {"color": "#ff7f0e", "desc": "(Strong Claim, Low Confidence)"},
    "Low SHAP, Low Unc": {"color": "#2ca02c", "desc": "(Boring but Reliable)"},
    "Low SHAP, High Unc": {"color": "#d62728", "desc": "(Uncertain & Uninformative)"},
}
plt.figure(figsize=(10, 7))
for label, info in categories.items():
    mask = quad_df["category"] == label
    plt.scatter(quad_df.loc[mask, "shap_mag"], quad_df.loc[mask, "unc_mag"], c=info["color"],
                label=f"{label} {info['desc']}", alpha=0.7, edgecolors="w", s=100)
plt.axvline(s_mid, color="black", linestyle="--", alpha=0.3)
plt.axhline(u_mid, color="black", linestyle="--", alpha=0.3)
plt.xlabel(r"SHAP Magnitude (Strength of Evidence in $)")
plt.ylabel(r"Total Uncertainty Impact ($)")
plt.title("The Trust Map: Evidence vs. Confidence", fontsize=14, fontweight="bold")
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
plt.grid(alpha=0.2)
plt.tight_layout()
plt.show()

## Sample one point from each quadrant and waterfall

Compare the "Gold Standard" (High SHAP, Low Uncertainty) house vs the "High Chaos" house across Price, Epistemic, and Aleatoric.

In [ ]:
try:
    row_HS_LU = quad_df[quad_df["category"] == "High SHAP, Low Unc"]["shap_mag"].idxmax()
    row_chaos = quad_df["unc_mag"].idxmax()
    cat_LU, cat_chaos = "Gold Standard (Confident)", "High Chaos (Unreliable)"
    print(f"Selected: Row {row_HS_LU} (Confident) and {row_chaos} (Chaos)")
except Exception:
    row_HS_LU = 0
    row_chaos = max_unc_idx
    cat_LU, cat_chaos = "Sample", "Most Uncertain"

def plot_surgical_comparison(idx, category_name):
    exps = {
        "PRICE DRIVERS ($)": sv_results.get("mean"),
        "MODEL DOUBT (Epistemic $)": sv_results.get("epistemic"),
        "DATA NOISE (Aleatoric $)": sv_results.get("aleatoric"),
    }
    for title, values in exps.items():
        if values is None:
            continue
        exp_obj = shap.Explanation(
            values=values[idx],
            base_values=explainer_waterfall.expected_value if "PRICE" in title else 0,
            data=X_subset[idx],
            feature_names=feature_cols,
        )
        plt.figure(figsize=(10, 4))
        shap.plots.waterfall(exp_obj, max_display=7, show=False)
        plt.title(f"{title} | Row {idx} | {category_name}", fontsize=12, fontweight="bold", pad=20)
        plt.show()

print("--- Confident house ---")
plot_surgical_comparison(row_HS_LU, cat_LU)
print("--- High chaos house ---")
plot_surgical_comparison(row_chaos, cat_chaos)

## 95% interval summary and optional report export

In [ ]:
z = 1.96
lower = mu_mean_dollars - z * total_std_dollars
upper = mu_mean_dollars + z * total_std_dollars
hits = (y_test_raw >= lower) & (y_test_raw <= upper)
coverage = hits.mean()
print(f"95% Gaussian interval coverage: {coverage:.2%} (target: 95%)")

if os.environ.get("GCS_BUCKET") or os.environ.get("CI"):
    try:
        report_dir = os.path.join(os.path.dirname(os.path.abspath(".")), "reports")
    except Exception:
        report_dir = os.path.join(os.getcwd(), "reports")
    os.makedirs(report_dir, exist_ok=True)
    report_df = pd.DataFrame({
        "actual_price": y_test_raw,
        "predicted_price": mu_mean_dollars,
        "epistemic_std": epistemic_std_dollars,
        "aleatoric_std": aleatoric_std_dollars,
        "total_std": total_std_dollars,
        "in_95_interval": hits,
    })
    report_path = os.path.join(report_dir, "uncertainty_report.csv")
    report_df.to_csv(report_path, index=False)
    print(f"Saved report to {report_path}")